In [1]:
# add local directory to import path
import os
import sys
module_path = os.path.abspath(os.path.join('.'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
#### --- Standard imports ------
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from IPython.display import HTML

# local import
import nflplot

In [3]:
from importlib import reload
reload(nflplot)

<module 'nflplot' from 'c:\\Users\\pschl\\Documents\\data_science\\kaggle_nfl_bdb_2024\\nflplot.py'>

**This notebook assumes that the source data is in a "csv" subfolder relative to this notebook**

In [4]:
week = 1
track_df = pd.read_csv(f'csv/tracking_week_{week}.csv')
play_df = pd.read_csv('csv/plays.csv')
game_df = pd.read_csv('csv/games.csv')
player_df= pd.read_csv('csv/players.csv')
tackle_df = pd.read_csv('csv/tackles.csv')

In [5]:
solo_tackle_plays = tackle_df.query('tackle==1').loc[:, ['gameId','playId']].drop_duplicates()
len(solo_tackle_plays)

9918

In [6]:
print('Number of plays in week 1 with a solo tackle:')
len(play_df.merge(game_df.loc[game_df.week==1, 'gameId'], on='gameId', how='inner')
    .merge(solo_tackle_plays, on=['gameId','playId'], how='inner')
    )

Number of plays in week 1 with a solo tackle:


1213

Get the frame 10 before the end for each tracking play (event coordinators have said that the recordings end 5 frames after the end of the play)

In [7]:
pm10_df = (track_df.groupby(['gameId','playId','frameId']).head(1)
           .sort_values(['gameId','playId','frameId'], ascending=[True, True, True])
           .groupby(['gameId','playId'])
           .tail(10)
           .loc[:, ['gameId','playId','frameId']]
           )
pm10_df

,gameId,playId,frameId
12,2022090800,56,13
13,2022090800,56,14
14,2022090800,56,15
15,2022090800,56,16
16,2022090800,56,17
...,...,...,...
1406268,2022091200,3826,49
1406269,2022091200,3826,50
1406270,2022091200,3826,51
1406271,2022091200,3826,52


In [8]:
print('Events during solo tackle plays, in the last 10 frames of the play:')
(
    track_df.groupby(['gameId','playId','frameId']).head(1)
    .merge(pm10_df, on=['gameId','playId','frameId'], how='inner')
    .merge(solo_tackle_plays, on=['gameId','playId'], how='inner')
    .loc[:, 'event']
    .value_counts())

Events during solo tackle plays, in the last 10 frames of the play:


event
tackle                      974
out_of_bounds               203
first_contact               199
pass_outcome_caught          73
fumble                       25
pass_arrived                 20
qb_slide                     14
fumble_defense_recovered      1
Name: count, dtype: int64

Check if there is any overlap of these events during the same play (event count > 1)

In [9]:
(
    track_df.groupby(['gameId','playId','frameId']).head(1).loc[~track_df.event.isna(), ['gameId','playId','frameId','event']]
    .merge(solo_tackle_plays, on=['gameId','playId'], how='inner')
    .query('event.isin(["tackle","out_of_bounds","fumble","qb_slide","fumble_defense_recovered"])')
    .groupby(['gameId','playId']).count().reset_index().sort_values('event', ascending=False)
)

,gameId,playId,frameId,event
805,2022091109,1827,3,3
868,2022091110,1113,2,2
555,2022091106,442,2,2
806,2022091109,1886,1,1
812,2022091109,2267,1,1
...,...,...,...,...
403,2022091104,1871,1,1
402,2022091104,1847,1,1
401,2022091104,1826,1,1
400,2022091104,1736,1,1


In [10]:
(
    track_df.groupby(['gameId','playId','frameId']).head(1).loc[~track_df.event.isna(), ['gameId','playId','frameId','event']]
    .query('gameId==2022091106 & playId==442')
)

,gameId,playId,frameId,event
680710,2022091106,442,3,pass_arrived
680713,2022091106,442,6,pass_outcome_caught
680728,2022091106,442,21,fumble
680730,2022091106,442,23,out_of_bounds


Of the 3 plays in week 1 with overlap of these events:
* All involve a fumble
* One fumble recovered by the defense where the tackle is actually the defender who recovered the ball
* One play is fumbling out of bounds without a tackle

Will need to probably evaluate first_contact event as the evaluation point, or an equivalent if the defender who tackles is not the first contact defender. Tackle appears to be when the play is over.

In [15]:
tackle_df.groupby(['gameId','playId']).sum().drop(columns='nflId').query('tackle >= 1').sort_values('tackle', ascending=False)

tackle  assist  forcedFumble  pff_missedTackle
gameId     playId                                                
2022091107 1879         2       1             1                 0
2022090800 56           1       0             0                 0
2022101610 1351         1       0             0                 1
           1044         1       0             0                 0
           1065         1       0             0                 0
...                   ...     ...           ...               ...
2022092511 3528         1       0             0                 1
           3549         1       0             0                 1
           3584         1       0             0                 0
           3660         1       0             0                 0
2022110700 3787         1       0             0                 0

[9918 rows x 4 columns]

In [19]:
tackle_df.query('gameId==2022091107 & playId==1879').merge(player_df, on='nflId', how='inner')

,gameId,playId,nflId,tackle,assist,forcedFumble,pff_missedTackle,height,weight,birthDate,collegeName,position,displayName
0,2022091107,1879,44854,1,0,0,0,6-1,195,1996-09-08,Utah,FS,Marcus Williams
1,2022091107,1879,54479,1,0,0,0,6-4,220,NaN,Notre Dame,FS,Kyle Hamilton
2,2022091107,1879,43694,0,1,1,0,6-0,340,1992-11-06,Samford,NT,Michael Pierce


In [18]:
play_df.query('gameId==2022091107 & playId==1879').playDescription.iloc[0]

'(1:16) (Shotgun) J.Flacco pass short right to T.Conklin to BLT 21 for 6 yards (M.Williams, M.Pierce). FUMBLES (M.Pierce), touched at BLT 25, recovered by NYJ-G.Wilson at BLT 27. G.Wilson to BLT 27 for no gain (K.Hamilton).'

gameId 2022091107, playId 1879 should be thrown out since this is 2 tackles due to the fumble so it would be confusing for the automated analysis. This edge case is not worth solving around to demonstrate a proof of concept.

In [32]:
tackle_df.groupby(['gameId','playId']).sum().drop(columns='nflId').query('tackle == 0 & assist > 0').sort_values('assist', ascending=False)

tackle  assist  forcedFumble  pff_missedTackle
gameId     playId                                                
2022090800 146          0       2             0                 0
2022101610 1327         0       2             0                 0
2022101611 576          0       2             0                 0
           485          0       2             0                 0
2022101610 4090         0       2             0                 0
...                   ...     ...           ...               ...
2022092900 2553         0       2             0                 0
           2375         0       2             0                 0
           2354         0       2             0                 0
           2275         0       2             0                 0
2022110700 3630         0       2             0                 0

[2004 rows x 4 columns]

~15% of plays that have tackle = 1 have assist = 1. May want to put a flag on the play if assist > 0 for the play for future filtering if desired to isolate solo tackles